<a href="https://colab.research.google.com/github/Tanmoy-prog/Deep-Learning/blob/main/Hyperparameter_tuning_in_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [ ]:
fashion_mist = keras.datasets.fashion_mnist

In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 102kB 4.3MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-cp37-none-any.whl size=9569 sha256=cdf07b633d39a2a220353f386d8fdec808b2014e6b5148a330dc6b86ae9b23d5
  Stored in directory: /root/.cache/pip/wheels/95/99/a8/6b116b0f69be60cc475d1cd36680f3a09f284d86655bb99d93
Successfully built kt-legacy


In [ ]:
fashion_mist


<module 'tensorflow.keras.datasets.fashion_mnist' from '/usr/local/lib/python3.7/dist-packages/tensorflow/keras/datasets/fashion_mnist/__init__.py'>

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mist.load_data()

In [ ]:
train_images = train_images/255.0
test_images = test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
len(train_images)

60000

In [ ]:
len(test_images)

10000

In [ ]:
test_images[0].shape

(28, 28)

In [ ]:
train_images= train_images.reshape(len(train_images),28,28,1)


In [ ]:
test_images= test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        64032     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                1239120   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 1,304,762
Trainable params: 1,304,762
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 421s 249ms/step - loss: 0.0154 - accuracy: 0.9947 - val_loss: 0.6765 - val_accuracy: 0.9115
Epoch 5/10
1688/1688 [==============================] - 432s 256ms/step - loss: 0.0152 - accuracy: 0.9951 - val_loss: 0.6940 - val_accuracy: 0.9132
Epoch 6/10
1688/1688 [==============================] - 423s 251ms/step - loss: 0.0129 - accuracy: 0.9961 - val_loss: 0.6945 - val_accuracy: 0.9108
Epoch 7/10
1688/1688 [==============================] - 423s 251ms/step - loss: 0.0119 - accuracy: 0.9960 - val_loss: 0.7851 - val_accuracy: 0.9140
Epoch 8/10
1688/1688 [==============================] - 430s 255ms/step - loss: 0.0130 - accuracy: 0.9960 - val_loss: 0.7943 - val_accuracy: 0.9073
Epoch 9/10
1688/1688 [==============================] - 426s 252ms/step - loss: 0.0121 - accuracy: 0.9960 - val_loss: 0.7731 - val_accuracy: 0.9125
Epoch 10/10
1688/1688 [==============================] - 417s 247ms/step - loss: 0.0096 - accuracy: 0.9970 - val